In [1]:
import pandas as pd
import numpy as np

# Cargar el archivo CSV en un DataFrame de pandas
data = pd.read_csv("/content/sample_data/MXM00076683.csv")

# 1. Eliminación de duplicados
data = data.drop_duplicates()

# 2. Conversión de tipos de datos
# Convertir columnas de fecha al tipo datetime (ajusta según el nombre de tus columnas de fecha)
date_columns = ['DATE']  # Reemplazar con los nombres de tus columnas de fecha
for column in date_columns:
    data[column] = pd.to_datetime(data[column], errors='coerce')

# Convertir columnas numéricas al tipo float
numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns
data[numeric_columns] = data[numeric_columns].astype(float)

# 3. Manejo de valores extremos en columnas de temperatura y precipitación
# Definir umbrales para columnas relacionadas con la temperatura (TMAX, TMIN, TAVG) y precipitación (PRCP)
def handle_outliers(column, lower, upper):
    return column.apply(lambda x: np.nan if x < lower or x > upper else x)

data['TMAX'] = handle_outliers(data['TMAX'], -50, 50)
data['TMIN'] = handle_outliers(data['TMIN'], -50, 50)
data['TAVG'] = handle_outliers(data['TAVG'], -50, 50)
data['PRCP'] = handle_outliers(data['PRCP'], 0, 1000)  # Ajustar el umbral de PRCP según los datos

# 4. Remplazo de valores inconsistentes en columnas de texto
# Normalizar valores de texto eliminando caracteres especiales, espacios en blanco y usando minúsculas
text_columns = ['NAME']  # Reemplazar con los nombres de columnas de texto
for column in text_columns:
    data[column] = data[column].str.replace(r"[^a-zA-Z0-9 ]", "", regex=True).str.lower().str.strip()

# 5. Manejo de valores nulos (reemplazo con la media en columnas numéricas)
for column in numeric_columns:
    mean_value = data[column].mean()
    data[column].fillna(mean_value, inplace=True)

# 6. Eliminación de columnas innecesarias
unnecessary_columns = ['PRCP_ATTRIBUTES', 'SNWD_ATTRIBUTES', 'TMAX_ATTRIBUTES', 'TMIN_ATTRIBUTES', 'TAVG_ATTRIBUTES']  # Ajusta según tus necesidades
data = data.drop(columns=unnecessary_columns, errors='ignore')

# 7. Normalización de datos sin paralelización
# Normalizamos cada columna numérica entre 0 y 1
def normalize_column(column):
    min_val = column.min()
    max_val = column.max()
    return (column - min_val) / (max_val - min_val)

# Normalizar cada columna numérica y crear nuevas columnas escaladas
for column in numeric_columns:
    data[column + "_scaled"] = normalize_column(data[column])

# Mostrar algunas filas del DataFrame limpio y normalizado
print(data.head())

# Guardar el DataFrame limpio y normalizado en un nuevo archivo CSV si es necesario
data.to_csv("archivo_normalizado_l.csv", index=False)

<ipython-input-1-62feda228610>:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].fillna(mean_value, inplace=True)


       STATION       DATE  LATITUDE  LONGITUDE  ELEVATION  \
0  MXM00076683 1961-10-01   19.3167   -98.2333     2229.9   
1  MXM00076683 1961-10-02   19.3167   -98.2333     2229.9   
2  MXM00076683 1961-10-03   19.3167   -98.2333     2229.9   
3  MXM00076683 1961-10-04   19.3167   -98.2333     2229.9   
4  MXM00076683 1961-10-05   19.3167   -98.2333     2229.9   

                             NAME  PRCP        SNWD  TMAX      TMIN  TAVG  \
0  tlaxcala de xicontecatl dge mx   0.0  915.857143   NaN  28.17121   NaN   
1  tlaxcala de xicontecatl dge mx  64.0  915.857143   NaN  28.17121   NaN   
2  tlaxcala de xicontecatl dge mx   0.0  915.857143   NaN  28.17121   NaN   
3  tlaxcala de xicontecatl dge mx   0.0  915.857143   NaN  28.17121   NaN   
4  tlaxcala de xicontecatl dge mx   0.0  915.857143   NaN  28.17121   NaN   

   LATITUDE_scaled  LONGITUDE_scaled  ELEVATION_scaled  PRCP_scaled  \
0              NaN               NaN               NaN     0.000000   
1              NaN          